In [1]:
import pandas as pd

# Load your data
df = pd.read_excel('car_fuel_efficiency.xlsx')

# Quick check
print(df.head())
print(df.isnull().sum())


   engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                  170            3.0       159.0     3413.433759   
1                  130            5.0        97.0     3149.664934   
2                  170            NaN        78.0     3079.038997   
3                  220            4.0         NaN     2542.392402   
4                  210            1.0       140.0     3460.870990   

   acceleration  model_year  origin fuel_type         drivetrain  num_doors  \
0          17.7        2003  Europe  Gasoline    All-wheel drive        0.0   
1          17.8        2007     USA  Gasoline  Front-wheel drive        0.0   
2          15.1        2018  Europe  Gasoline  Front-wheel drive        0.0   
3          20.2        2009     USA    Diesel    All-wheel drive        2.0   
4          14.4        2009  Europe  Gasoline    All-wheel drive        2.0   

   fuel_efficiency_mpg  
0            13.231729  
1            13.688217  
2            14.246341  
3         

In [2]:
df = df.fillna(0)

In [3]:
from sklearn.model_selection import train_test_split

# Separate the target variable
y = df['fuel_efficiency_mpg']
X = df.drop(columns=['fuel_efficiency_mpg'])

# Split: 60% train, 40% temp (which will later be split into val/test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1)

# Split the remaining 40% equally into validation and test sets (20% each overall)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Check sizes
print(len(X_train), len(X_val), len(X_test))


5822 1941 1941


In [4]:
from sklearn.feature_extraction import DictVectorizer

# Convert dataframes to lists of dictionaries
train_dicts = X_train.to_dict(orient='records')
val_dicts = X_val.to_dict(orient='records')
test_dicts = X_test.to_dict(orient='records')

# Initialize the vectorizer
dv = DictVectorizer(sparse=True)

# Fit on training data, transform all sets
X_train_dv = dv.fit_transform(train_dicts)
X_val_dv = dv.transform(val_dicts)
X_test_dv = dv.transform(test_dicts)


In [ ]:
#question 2
from sklearn.tree import DecisionTreeRegressor

# Train a decision tree regressor with depth = 1
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train_dv, y_train)


,criterion,'squared_error'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,1
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [6]:
feature_names = dv.get_feature_names_out()
feature_importances = dt.feature_importances_

# Get the feature with the highest importance
split_feature = feature_names[feature_importances.argmax()]
print("Feature used for the split:", split_feature)


Feature used for the split: vehicle_weight


In [7]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=10,
    random_state=1,
    n_jobs=-1
)

rf.fit(X_train_dv, y_train)


,n_estimators,10
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [8]:
y_pred = rf.predict(X_val_dv)


In [9]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE:", rmse)


Validation RMSE: 0.4607918868758161


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

n_estimators_list = range(10, 201, 10)
rmse_list = []

for n in n_estimators_list:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train_dv, y_train)
    y_pred = rf.predict(X_val_dv)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_list.append(rmse)
    print(f"n_estimators={n}, RMSE={rmse:.3f}")


    80


n_estimators=10, RMSE=0.461
n_estimators=20, RMSE=0.447
n_estimators=30, RMSE=0.441
n_estimators=40, RMSE=0.439
n_estimators=50, RMSE=0.437
n_estimators=60, RMSE=0.435
n_estimators=70, RMSE=0.436
n_estimators=80, RMSE=0.436
n_estimators=90, RMSE=0.436
n_estimators=100, RMSE=0.436
n_estimators=110, RMSE=0.435
n_estimators=120, RMSE=0.436
n_estimators=130, RMSE=0.435
n_estimators=140, RMSE=0.435
n_estimators=150, RMSE=0.435
n_estimators=160, RMSE=0.435
n_estimators=170, RMSE=0.435
n_estimators=180, RMSE=0.435
n_estimators=190, RMSE=0.435
n_estimators=200, RMSE=0.435


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

max_depth_list = [10, 15, 20, 25]
n_estimators_list = range(10, 201, 10)

results = []

for depth in max_depth_list:
    rmse_for_depth = []
    for n in n_estimators_list:
        rf = RandomForestRegressor(n_estimators=n, max_depth=depth, random_state=1, n_jobs=-1)
        rf.fit(X_train_dv, y_train)
        y_pred = rf.predict(X_val_dv)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_for_depth.append(rmse)
    mean_rmse = np.mean(rmse_for_depth)
    results.append((depth, mean_rmse))
    print(f"max_depth={depth}, mean RMSE={mean_rmse:.3f}")


max_depth=10, mean RMSE=0.436
max_depth=15, mean RMSE=0.438
max_depth=20, mean RMSE=0.438
max_depth=25, mean RMSE=0.438


In [12]:
best_depth, best_rmse = min(results, key=lambda x: x[1])
print(f"Best max_depth: {best_depth}, with mean RMSE={best_rmse:.3f}")


Best max_depth: 10, with mean RMSE=0.436


In [13]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1
)

rf.fit(X_train_dv, y_train)


,n_estimators,10
,criterion,'squared_error'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [14]:
# Get feature names from DictVectorizer
feature_names = dv.get_feature_names_out()

# Get importance values
importances = rf.feature_importances_

# Combine into a dictionary and sort
feature_importance_dict = dict(zip(feature_names, importances))
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Most important feature
most_important_feature = sorted_features[0][0]
print("Most important feature:", most_important_feature)


Most important feature: vehicle_weight


In [15]:
import xgboost as xgb

# DMatrix for training and validation
dtrain = xgb.DMatrix(X_train_dv, label=y_train)
dval = xgb.DMatrix(X_val_dv, label=y_val)


In [16]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1
}

# Watchlist to track performance
watchlist = [(dtrain, 'train'), (dval, 'val')]


In [17]:
bst_03 = xgb.train(params=xgb_params, 
                   dtrain=dtrain, 
                   num_boost_round=100, 
                   evals=watchlist, 
                   early_stopping_rounds=10, 
                   verbose_eval=False)

# Get RMSE on validation
rmse_03 = bst_03.eval(dval)
print("RMSE with eta=0.3:", rmse_03)


RMSE with eta=0.3: [0]	eval-rmse:0.42667345104720683


In [18]:
xgb_params['eta'] = 0.1

bst_01 = xgb.train(params=xgb_params, 
                   dtrain=dtrain, 
                   num_boost_round=100, 
                   evals=watchlist, 
                   early_stopping_rounds=10, 
                   verbose_eval=False)

# Get RMSE on validation
rmse_01 = bst_01.eval(dval)
print("RMSE with eta=0.1:", rmse_01)


RMSE with eta=0.1: [0]	eval-rmse:0.41651146160407321
